In [22]:
# Importing Libraries
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers
from sklearn.impute import KNNImputer
from sklearn.metrics.pairwise import nan_euclidean_distances


In [3]:
# Import datasets
missingdata1 = pd.read_csv('input/MissingData1.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
missingdata2 = pd.read_csv('input/MissingData2.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
missingdata3 = pd.read_csv('input/MissingData3.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
missing_data = [missingdata1, missingdata2, missingdata3]

In [4]:
# Counting the missing values
def count_na(data):
    print("Missing values in the data:")
    for i in range(len(data)):
        missing_val_count = data[i].isnull().sum().sum()
        print( "MISSING DATA  " + str(i+1) + ": " + str(missing_val_count))
        # for j in range(data[i].shape[1]):
        #     missing_val_count = data[i].iloc[:,j].isnull().sum()
        #     perc = (missing_val_count/data[i].shape[0])*100
        #     perc = round(perc, 2)
        #     print( "\tCOLUMN " + str(j+1) + ": " + str(missing_val_count) + " (" + str(perc) + "%)")
            

count_na(missing_data)

Missing values in the data:
MISSING DATA  1: 118
MISSING DATA  2: 3762
MISSING DATA  3: 17752


1. choose missing val to fill
2. select the other vals in that row
3. choose the number of neighbours: k = n/2
4. calc euclidean distance from all the other rows
5. select k nearest neighbours
6. find the mean of the k nearest neighbours values in the row

In [54]:
def knn_imputation(data,n_neighbors = 2):
    count = 0
    ## Where are the null values in the data set
    indexes_nan = np.argwhere(np.isnan(data))
    n_col = data.shape[1]
    n_row = data.shape[0]
    final_data = data.copy()
    for index in indexes_nan:
        count = count + 1 
        distances = []
        row = index[0]
        col = index[1]
        x = data[row]
        for i in range(n_row):
            if(i==row):
                continue
            dist = nan_euclidean_distances(x.reshape(1,-1),data[i].reshape(1,-1))[0][0]
            ## or u can also use the above distance function, both produce same answer
            ## dist = nan_distance(x,data[i])
            distances.append([dist, data[i][col]])
        ## key=lambda x: x[0]  here x:x[0] signifies that we will sort this list using 
        ## 1st(in python indexing starts from 0) column i.e. distance
        distances.sort(key=lambda x: x[0])
         ## Considering k closest points
        distances = np.array(distances)
        neighbors = []
        for i in range(len(distances)):
            if(np.isnan(distances[i,1])):
                continue
            neighbors.append(distances[i,1])
            if(len(neighbors)==n_neighbors):  
                break
        ## replacing the nan with the mean of the closest points
        adjusted_value = np.mean(neighbors)
        final_data[row][col] = adjusted_value
    
    return(final_data)
def optimized(data,n_neighbors = 2):
    count = 0
    ## Where are the null values in the data set
    indexes_nan = np.argwhere(np.isnan(data))
    final_data = data.copy()
    ## it will produce a distance matrix with row in the dataset
    dist = nan_euclidean_distances(data)
    for index in indexes_nan:
        count = count + 1 
        row = index[0]
        col = index[1]
        ## Sorting the distance matrix for the particular row and getting top n values
        sort_indxs = dist[row].argsort()
        close_val = []
        for i in sort_indxs:
            if(np.isnan(data[i,col])):
                continue
            close_val.append(data[i,col])
            if(len(close_val)==n_neighbors):
                break
        adjusted_value = np.mean(close_val)
        final_data[row][col] = adjusted_value
    return(final_data)

In [53]:

# for eaching missing value, perform knn impute and write to file
for i in range(len(missing_data)):
    final = knn_imputation(missing_data[i].values,n_neighbors =  242/2)
    final = pd.DataFrame(final)
    final.to_csv('output/MissingData'+str(i+1)+'.txt', sep='\t', index=False, header=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


FileNotFoundError: [Errno 2] No such file or directory: 'output/MissingData1.txt'